In [1]:
%cd /home/kyuholee/SPD_ver_3
import torch
from B_data.iii_datamodule import stockDataModule
from omegaconf import OmegaConf

name = 'transformer_classification/exp_4'
n = 4

data_config = OmegaConf.load('/content/drive/MyDrive/Auto_trading/SPD_ver_3/A_config/data_config.yaml')
config = OmegaConf.load(f'{data_config.path.config_path}/{name}/setting{n}.yaml')

A = stockDataModule(config, data_config)


[Errno 2] No such file or directory: '/home/kyuholee/SPD_ver_3'
/content/drive/MyDrive/Auto_trading/SPD_ver_3/G_jupyter


ModuleNotFoundError: No module named 'B_data'

In [ ]:
A.setup()
inp = next(iter(A.train_dataloader()))

In [ ]:
print(inp[0].shape)

torch.Size([128, 100, 29])


In [1]:
%cd /home/kyuholee/SPD_ver_3
import torch
from C_model.i_transformer import TransformerModule
from omegaconf import OmegaConf

name = 'exp_4'
n = 4

data_config = OmegaConf.load('/home/kyuholee/SPD_ver_3/A_config/data_config.yaml')
config = OmegaConf.load(f'{data_config.path.config_path}/{name}/setting{n}.yaml')

layer = TransformerModule(config)

inp = torch.randn(128, 100, 29)

output = layer(inp)
#output = torch.nn.functional.softmax(output, dim=1)

/home/kyuholee/SPD_ver_3
torch.Size([128, 100, 256])
torch.Size([128, 256])
torch.Size([128, 3])


In [2]:
%cd /content/drive/MyDrive/Auto_trading/SPD_ver_3
import torch
from C_model.ii_lstm import LSTMEncoder
from omegaconf import OmegaConf

args = {
    'ninp': 29,
    'nhid': 256,
    'nlayers': 3,
    'dropout': 0.1,
    'n_class':3
}

inp = torch.randn(128, 100, 29)

L = LSTMEncoder(args)

out = L(inp)

[Errno 2] No such file or directory: '/home/kyuholee/SPD_ver_3'
/content/drive/MyDrive/Auto_trading/SPD_ver_3/G_jupyter


ModuleNotFoundError: No module named 'C_model'

In [2]:
print(out.shape)

torch.Size([128, 3])


In [27]:

print(output[:10])
print(torch.nn.functional.softmax(output[0:10], dim=1))

print(torch.nn.functional.cross_entropy(output, inp[1]))

tensor([[-0.0365, -0.0371, -0.0525],
        [-0.0168, -0.0597, -0.0239],
        [-0.0290, -0.0912, -0.0802],
        [-0.0337, -0.0534,  0.0112],
        [-0.0084, -0.0523, -0.0337],
        [-0.0132, -0.0708, -0.0358],
        [ 0.0084, -0.0947, -0.0220],
        [-0.0576, -0.0702, -0.0324],
        [ 0.0200, -0.0572, -0.0310],
        [-0.0574, -0.0518, -0.0596]], grad_fn=<SliceBackward0>)
tensor([[0.3352, 0.3350, 0.3299],
        [0.3389, 0.3246, 0.3365],
        [0.3460, 0.3252, 0.3288],
        [0.3304, 0.3240, 0.3456],
        [0.3411, 0.3264, 0.3325],
        [0.3423, 0.3231, 0.3346],
        [0.3482, 0.3141, 0.3378],
        [0.3319, 0.3277, 0.3403],
        [0.3477, 0.3219, 0.3304],
        [0.3330, 0.3348, 0.3322]], grad_fn=<SoftmaxBackward0>)
tensor(1.0978, grad_fn=<NllLossBackward0>)


In [41]:
import torchmetrics

acc = torchmetrics.Accuracy(task="multiclass", num_classes=3)

accuracy = acc(output, inp[1])

print(accuracy)
print(inp[1])

tensor(0.3203)
tensor([0, 2, 0, 2, 1, 1, 0, 0, 0, 1, 1, 1, 2, 1, 1, 1, 1, 1, 0, 2, 1, 0, 0, 1,
        1, 2, 2, 1, 0, 2, 1, 0, 0, 1, 2, 2, 1, 0, 0, 1, 1, 0, 2, 1, 2, 1, 1, 1,
        1, 1, 2, 2, 1, 0, 0, 1, 0, 1, 1, 0, 0, 2, 2, 2, 1, 2, 1, 1, 2, 2, 2, 2,
        1, 0, 2, 0, 2, 0, 0, 1, 0, 1, 2, 2, 0, 2, 1, 0, 2, 0, 2, 1, 0, 2, 0, 0,
        1, 1, 0, 1, 1, 0, 2, 0, 2, 0, 1, 0, 2, 2, 2, 1, 1, 0, 0, 2, 2, 2, 2, 2,
        2, 1, 1, 2, 1, 1, 0, 0])


In [5]:
from B_data.ii_db_management import db_coin_list

from omegaconf import OmegaConf

data_config = OmegaConf.load('/home/kyuholee/SPD_ver_3/A_config/data_config.yaml')

print(len(db_coin_list(data_config)))

69


In [3]:
%cd /home/kyuholee/SPD_ver_3
from B_data.iii_datamodule import BackTestDataModule
from C_model.i_transformer import TransformerModule

from omegaconf import OmegaConf
import lightning.pytorch as pl

config = OmegaConf.load('/home/kyuholee/SPD_ver_3/A_config/exp_4/setting3.yaml')
data_config = OmegaConf.load('/home/kyuholee/SPD_ver_3/A_config/data_config.yaml')

A = BackTestDataModule(config, data_config, 'BTCUSDT', [2023, 6, 1, 10], [2023, 7, 1, 10])
A.setup()


model = TransformerModule.load_from_checkpoint(f'{data_config.path.result_path}/{config.name}/setting{config.num}.ckpt')

trainer = pl.Trainer(logger=None, use_distributed_sampler=False)
predictions = trainer.predict(model=model, datamodule=A)


/home/kyuholee/SPD_ver_3


Trainer will use only 1 of 4 GPUs because it is running inside an interactive / notebook environment. You may try to set `Trainer(devices=4)` but please note that multi-GPU inside interactive / notebook environments is considered experimental and unstable. Your mileage may vary.
GPU available: True (cuda), used: True
TPU available: False, using: 0 TPU cores
IPU available: False, using: 0 IPUs
HPU available: False, using: 0 HPUs
/opt/conda/lib/python3.10/site-packages/lightning/pytorch/trainer/connectors/logger_connector/logger_connector.py:67: Starting from v1.9.0, `tensorboardX` has been removed as a dependency of the `lightning.pytorch` package, due to potential conflicts with other packages in the ML ecosystem. For this reason, `logger=True` will use `CSVLogger` as the default logger, unless the `tensorboard` or `tensorboardX` packages are found. Please `pip install lightning[extra]` or one of them to enable TensorBoard support by default
LOCAL_RANK: 0 - CUDA_VISIBLE_DEVICES: [0,1,2

Predicting: |          | 0/? [00:00<?, ?it/s]

In [6]:
print(predictions[0])

(tensor([100]), 1)


In [42]:
import numpy as np

k = np.array([1,2,3,4])

print(list(k))

[1, 2, 3, 4]
